In [1]:
from dataset import CocoDataset
from torch.utils.data import DataLoader
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

In [6]:
import onnxruntime
# 测试onnxtime的output
providers = ['CUDAExecutionProvider']
onnxruntime_model_path = "/home/geng/tinyml/ppq/benchmark/detection/OpenVino_output/Retinanet-wo-ORT-INT8.onnx"
sess = onnxruntime.InferenceSession(path_or_bytes=onnxruntime_model_path, providers=providers)
input_placeholder_name = sess.get_inputs()[0].name
outputnames = [x.name for x in sess.get_outputs()]



In [7]:
import re
outputnames.sort(key=lambda x:int(re.findall("\d+",x)[0]))
outputnames

<>:2: DeprecationWarning: invalid escape sequence \d
<>:2: DeprecationWarning: invalid escape sequence \d
/tmp/ipykernel_1239931/2631274711.py:2: DeprecationWarning: invalid escape sequence \d
  outputnames.sort(key=lambda x:int(re.findall("\d+",x)[0]))


['PPQ_Variable_2363',
 'PPQ_Variable_2423',
 'PPQ_Variable_2483',
 'PPQ_Variable_2543',
 'PPQ_Variable_2603',
 'PPQ_Variable_2609',
 'PPQ_Variable_2615',
 'PPQ_Variable_2621',
 'PPQ_Variable_2627',
 'PPQ_Variable_2633']

In [2]:
ann_file = "/home/geng/fiftyone/coco-2017/validation/labels.json"
# dataset_type = 'CocoDataset'  # 数据集类型，这将被用来定义数据集。
data_root = '/home/geng/fiftyone/coco-2017/validation/data/'  # 数据的根路径。
batch_size = 1
# input_size = (1216,800)
input_size = (480,640)

img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)


test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=input_size,
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=False),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
# dataset = CocoDataset(ann_file=ann_file, pipeline=test_pipeline,data_root=data_root)

In [3]:
# from mmcv.parallel import collate
# batch_size = 1
# data_loader = DataLoader(
#         dataset,
#         batch_size=batch_size,
#         collate_fn=collate
#         )

In [3]:
from dataset import build_dataset
from mmcv.parallel import collate
dataset = build_dataset(ann_file=ann_file,data_root=data_root,input_size=input_size)
dataloader = DataLoader(dataset,batch_size=1,collate_fn=collate)

loading annotations into memory...
Done (t=0.74s)
creating index...
index created!


In [17]:
import cfg
calib_dataloader = [dataset[i]["img"][0].unsqueeze(0) for i in range(cfg.CALIBRATION_NUM)]


      ____  ____  __   ____                    __              __
     / __ \/ __ \/ /  / __ \__  ______ _____  / /_____  ____  / /
    / /_/ / /_/ / /  / / / / / / / __ `/ __ \/ __/ __ \/ __ \/ /
   / ____/ ____/ /__/ /_/ / /_/ / /_/ / / / / /_/ /_/ / /_/ / /
  /_/   /_/   /_____\___\_\__,_/\__,_/_/ /_/\__/\____/\____/_/




In [19]:
calib_dataloader[0].shape

torch.Size([1, 3, 480, 640])

In [5]:
from tqdm import tqdm
import onnxruntime
import torch
import numpy as np
#  onnx推理过程
def onnxruntime_inference(data_loader,onnxruntime_model_path):
    sess = onnxruntime.InferenceSession(path_or_bytes=onnxruntime_model_path, providers=['CUDAExecutionProvider'])
    input_placeholder_name = sess.get_inputs()[0].name
    outputnames = [x.name for x in sess.get_outputs()]
    outputs = []
    for x in tqdm(data_loader):
        input_tensor = x["img"][0]
        img_metas = x["img_metas"][0].data[0][0]
        output = sess.run(input_feed={input_placeholder_name: input_tensor.cpu().numpy()},output_names=outputnames)
        img_metas["output"] = output
        outputs.append(img_metas)
    return outputs

In [6]:
x = next(iter(dataloader))

In [7]:
x["img"][0].shape

torch.Size([1, 3, 480, 640])

In [8]:
"""
将任意模型的输出转换为常规输出
[N,result]  N为样本数
result: [C,[M,5]] 其中C为类别,M为bbox数目，5为左上角、右下角坐标与置信分数。
并且根据尺度进行复原
"""
# 模型后处理过程
def post_process(model_name,outputs):
    if model_name == "Retinanet":
        results = []
        for output in outputs:
            img_scale = output["scale_factor"] 
            labels,bboxs = output["output"]
            result = [[] for _ in range(len(dataset.cat_ids))]
            bboxs,labels = bboxs[0],labels[0]
            for i,label in enumerate(labels):
                bboxs[i][:4] = bboxs[i][:4] / img_scale  # 进行bbox的尺寸复原
                result[label].append(bboxs[i])
            results.append(result)
        return results
    elif model_name == "Retinanet-wo":
        pass

In [8]:
from inference import onnxruntime_inference
# onnxruntime_model_path = "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-wo-FP32.onnx"
onnxruntime_model_path = "/home/geng/tinyml/ppq/benchmark/detection/OpenVino_output/Retinanet-wo-ORT-INT8.onnx"
dataloader = [next(iter(dataloader))]
outputs = onnxruntime_inference(dataloader,onnxruntime_model_path)
# results = post_process("retinanet-end",outputs)

  0%|          | 0/1 [00:00<?, ?it/s]

infering on coco dataset....


100%|██████████| 1/1 [00:00<00:00,  1.25it/s]


In [5]:
conf,bbox = outputs[0]["output"][0],outputs[0]["output"][5]

In [5]:
output = outputs[0]["output"]
for t in output:
    x,y,z,u  =  t.shape
    print(x*y*z*u,t.shape)

172800 (1, 36, 60, 80)
43200 (1, 36, 30, 40)
10800 (1, 36, 15, 20)
2880 (1, 36, 8, 10)
720 (1, 36, 4, 5)
3456000 (1, 720, 60, 80)
864000 (1, 720, 30, 40)
216000 (1, 720, 15, 20)
57600 (1, 720, 8, 10)
14400 (1, 720, 4, 5)


In [9]:
from utils import retinanet_postprocess
import torch
class_num = 80
output = outputs[0]
img_scale = output["scale_factor"] 
input_size = output["img_shape"][:2] 
cls_heads,reg_heads = output["output"][:5],output["output"][5:]
result = [[] for _ in range(class_num)]

if reg_heads[0].shape[1] > cls_heads[0].shape[1]:
    # 执行ppq后，可能出现outputs通道顺序改变的情况
    # 正常reg_head.shape = (bs,9*4,h,w) cls_head.shape = (bs,9*80,h,w)
    # TODO:这里可能不严谨，只使用第二维大小来确定回归和分类输出，更好的应该只用output的名字
    # 已经分别在onnxruntime和trt实现了输出的指定。
    reg_heads,cls_heads = cls_heads,reg_heads
    # print(reg_heads.shape,cls_heads.shape)

scores, bboxs, labels = retinanet_postprocess(input_size,cls_heads,reg_heads)
bboxs,labels,scores = bboxs[0],labels[0].int(),scores.t()
print(bboxs.shape,labels.shape,scores.shape)
bboxs = torch.cat((bboxs,scores),dim=1)
for i,label in enumerate(labels):
    bboxs[i][:4] = bboxs[i][:4] / img_scale  # 进行bbox的尺寸复原
    result[label].append(bboxs[i])

torch.Size([100, 4]) torch.Size([100]) torch.Size([100, 1])


In [12]:
reg_heads[0].shape[1]

(1, 720, 60, 80)

In [15]:
result

[[tensor([3.8813e+02, 6.5603e+01, 4.9888e+02, 3.4966e+02, 4.8650e-01]),
  tensor([9.2645e-01, 2.5768e+02, 6.5128e+01, 3.0957e+02, 1.8524e-01]),
  tensor([3.8528e+02, 1.8040e+02, 4.7872e+02, 3.4587e+02, 1.4451e-01]),
  tensor([6.0518e-01, 2.2837e+02, 8.4929e+01, 3.0237e+02, 1.4294e-01]),
  tensor([5.5091e-01, 2.7719e+02, 7.7247e+01, 3.3615e+02, 1.3273e-01]),
  tensor([3.9357e+02, 7.4635e+01, 4.9601e+02, 1.9708e+02, 1.1561e-01]),
  tensor([2.4530e+02, 7.9997e+01, 4.8311e+02, 3.4617e+02, 1.1109e-01]),
  tensor([5.8882e+00, 1.9280e+02, 1.5058e+02, 2.9630e+02, 1.0597e-01]),
  tensor([3.5867e+02, 1.3255e+02, 3.9957e+02, 2.0507e+02, 9.7724e-02])],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [tensor([4.1692e+02, 1.1870e+02, 4.7188e+02, 2.0018e+02, 1.1377e-01])],
 [],
 [tensor([4.1901e+02, 1.2115e+02, 4.6986e+02, 1.9701e+02, 1.8524e-01]),
  tensor([4.0260e+02, 1.1703e+02, 4.7543e+02, 2.5569e+02, 1.7649e-01]),
  tensor([3.58

In [11]:
from utils import post_process
post_process("Retinanet-wo",outputs,80)

deal with outputs in Retinanet-wo type
720 36


[[[tensor([3.8789e+02, 6.5603e+01, 4.9912e+02, 3.4966e+02, 4.8650e-01]),
   tensor([1.9123e+00, 2.4161e+02, 7.1075e+01, 2.9854e+02, 1.8524e-01]),
   tensor([6.6876e-01, 2.6184e+02, 6.2964e+01, 3.0804e+02, 1.8524e-01]),
   tensor([3.8528e+02, 1.8040e+02, 4.7872e+02, 3.4587e+02, 1.5414e-01]),
   tensor([5.5091e-01, 2.7738e+02, 7.7247e+01, 3.3661e+02, 1.3273e-01]),
   tensor([1.3530e+00, 1.8255e+02, 9.9597e+01, 3.0577e+02, 1.2203e-01]),
   tensor([2.4803e+02, 7.8432e+01, 4.8482e+02, 3.4576e+02, 1.1875e-01]),
   tensor([3.9416e+02, 7.4635e+01, 4.9659e+02, 1.9708e+02, 1.1561e-01]),
   tensor([3.8691e+00, 2.2123e+02, 8.7875e+01, 2.8020e+02, 1.0502e-01])],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [tensor([4.2262e+02, 1.1774e+02, 4.6907e+02, 1.9156e+02, 1.1377e-01]),
   tensor([4.0456e+02, 1.0634e+02, 4.7913e+02, 2.0134e+02, 9.7724e-02])],
  [],
  [tensor([4.1915e+02, 1.2098e+02, 4.7024e+02, 1.97

In [22]:
shape = [t.shape for t in outputs[0]["output"]]
print(shape)

[(1, 36, 60, 80), (1, 36, 30, 40), (1, 36, 15, 20), (1, 36, 8, 10), (1, 36, 4, 5), (1, 720, 60, 80), (1, 720, 30, 40), (1, 720, 15, 20), (1, 720, 8, 10), (1, 720, 4, 5)]


In [131]:
#  box.py
def delta2box(deltas, anchors, size, stride):
    'Convert deltas from anchors to boxes'

    anchors_wh = anchors[:, 2:] - anchors[:, :2] + 1
    ctr = anchors[:, :2] + 0.5 * anchors_wh
    pred_ctr = deltas[:, :2] * anchors_wh + ctr
    pred_wh = torch.exp(deltas[:, 2:]) * anchors_wh

    m = torch.zeros([2], device=deltas.device, dtype=deltas.dtype)
    M = (torch.tensor([size], device=deltas.device, dtype=deltas.dtype) * stride - 1)
    clamp = lambda t: torch.max(m, torch.min(t, M))
    return torch.cat([
        clamp(pred_ctr - 0.5 * pred_wh),
        clamp(pred_ctr + 0.5 * pred_wh - 1)
    ], 1)

def decode(all_cls_head, all_box_head, stride=1, threshold=0.05, top_n=1000, anchors=None, rotated=False):
    'Box Decoding and Filtering'

    if rotated:
        anchors = anchors[0]
    num_boxes = 4 if not rotated else 6

    device = all_cls_head.device
    anchors = anchors.to(device).type(all_cls_head.type())
    num_anchors = anchors.size()[0] if anchors is not None else 1
    num_classes = all_cls_head.size()[1] // num_anchors
    height, width = all_cls_head.size()[-2:]

    batch_size = all_cls_head.size()[0]
    out_scores = torch.zeros((batch_size, top_n), device=device)
    out_boxes = torch.zeros((batch_size, top_n, num_boxes), device=device)
    out_classes = torch.zeros((batch_size, top_n), device=device)

    # Per item in batch
    for batch in range(batch_size):
        cls_head = all_cls_head[batch, :, :, :].contiguous().view(-1)
        box_head = all_box_head[batch, :, :, :].contiguous().view(-1, num_boxes)

        # Keep scores over threshold
        keep = (cls_head >= threshold).nonzero().view(-1)
        if keep.nelement() == 0:
            continue

        # Gather top elements
        scores = torch.index_select(cls_head, 0, keep)
        scores, indices = torch.topk(scores, min(top_n, keep.size()[0]), dim=0)
        indices = torch.index_select(keep, 0, indices).view(-1)
        classes = torch.div(indices, (width * height), rounding_mode='floor') % num_classes
        classes = classes.type(all_cls_head.type())
        

        # Infer kept bboxes
        x = indices % width
        y = torch.div(indices, width, rounding_mode='floor') % height
        a = torch.div(indices, (num_classes * height * width), rounding_mode='floor') 

        # print(box_head.shape,"-",(num_anchors, num_boxes, height, width))
        box_head = box_head.view(num_anchors, num_boxes, height, width)
        # print(f"a={a.dtype} y,x = {y.dtype,x.dtype}")
        boxes = box_head[a, :, y, x]

        if anchors is not None:
            grid = torch.stack([x, y, x, y], 1).type(all_cls_head.type()) * stride + anchors[a, :]
            boxes = delta2box(boxes, grid, [width, height], stride)

        out_scores[batch, :scores.size()[0]] = scores
        out_boxes[batch, :boxes.size()[0], :] = boxes
        out_classes[batch, :classes.size()[0]] = classes

    return out_scores, out_boxes, out_classes

In [145]:
def retinanet_postprocess(image_size, cls_heads, box_heads):
	# Inference post-processing
    anchors = {}
    decoded = []

    for cls_head, box_head in zip(cls_heads, box_heads):
	    # Generate level's anchors 
        cls_head,box_head = torch.from_numpy(cls_head),torch.from_numpy(box_head)
        stride = image_size[-1] // cls_head.shape[-1]
        if stride not in anchors:
            anchors[stride] = generate_anchors(stride, ratio_vals=[1.0, 2.0, 0.5],
                scales_vals=[4 * 2 ** (i / 3) for i in range(3)]) 
        decoded.append(decode(cls_head, box_head, stride,threshold=0.05, top_n=1000, anchors=anchors[stride]))

	# Perform non-maximum suppression
    decoded = [torch.cat(tensors, 1) for tensors in zip(*decoded)]
	# NMS threshold = 0.5
    scores, boxes, labels = nms(*decoded, nms=0.5, ndetections=100)
    return scores, boxes, labels

In [146]:
input_size = (1,3,480,640)
scores, boxes, labels = retinanet_postprocess(input_size,outputs[0]["output"][:5],outputs[0]["output"][5:])

In [148]:
boxes.shape,labels.shape

(torch.Size([1, 100, 4]), torch.Size([1, 100]))

In [155]:
boxes[0][0]

tensor([381.2623,  78.3648, 498.7371, 390.3024])

In [141]:
outputs[0]["img_shape"][::-1]

(3, 640, 480)

In [126]:
outputs[0]["output"][1].shape

(1, 720, 30, 40)

In [144]:
outputs[0]

{'filename': '/home/geng/fiftyone/coco-2017/validation/data/000000397133.jpg',
 'ori_filename': '000000397133.jpg',
 'ori_shape': (427, 640, 3),
 'img_shape': (480, 640, 3),
 'pad_shape': (480, 640, 3),
 'scale_factor': array([1.       , 1.1241218, 1.       , 1.1241218], dtype=float32),
 'flip': False,
 'flip_direction': None,
 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32),
  'std': array([58.395, 57.12 , 57.375], dtype=float32),
  'to_rgb': True},
 'output': [array([[[[0.00480121, 0.00261348, 0.00313526, ..., 0.00208086,
            0.0032661 , 0.01905698],
           [0.00176513, 0.00171477, 0.0040648 , ..., 0.00126189,
            0.00197899, 0.0129627 ],
           [0.00176001, 0.00220251, 0.0056954 , ..., 0.00152773,
            0.00229746, 0.01286143],
           ...,
           [0.00089306, 0.00053841, 0.0004192 , ..., 0.00129819,
            0.00242585, 0.01366252],
           [0.00251245, 0.00158536, 0.00139672, ..., 0.00167495,
            0.00280

In [4]:
from utils import (onnxruntime_inference2json,openvino_inference2json,
                    trt_inference2json,ppq_inference2json)


      ____  ____  __   ____                    __              __
     / __ \/ __ \/ /  / __ \__  ______ _____  / /_____  ____  / /
    / /_/ / /_/ / /  / / / / / / / __ `/ __ \/ __/ __ \/ __ \/ /
   / ____/ ____/ /__/ /_/ / /_/ / /_/ / / / / /_/ /_/ / /_/ / /
  /_/   /_/   /_____\___\_\__,_/\__,_/_/ /_/\__/\____/\____/_/




/home/geng/anaconda3/envs/ppq/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/home/geng/anaconda3/envs/ppq/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/geng/anaconda3/envs/ppq/lib/python3.8/site-packages/pandas/compat/numpy/function.py:125: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(_np_version) >= LooseVersion("1.17.0"):
/home/geng/anaconda3/envs/ppq/lib/python3.8/site-packages/pandas/compat/_optional.py:117: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:


In [5]:
# 测试opevino推理
model_name = "Retinanet"
model_path = "/home/geng/tinyml/ppq/benchmark/detection/OpenVino_output/Retinanet-OpenVino-INT8.onnx"
openvino_inference2json(dataset=dataset,model_name=model_name,model_path=model_path,
    batch_size=input_size[0],class_num=80,
    outfile_prefix=model_path[:-5],
    device="cuda")

: 

: 

In [15]:
dataset.results2json(results=results,outfile_prefix="./retinanet-trt-ort")  # 结果统一保存为coco json

{'bbox': './result/retinanet-trt-ort.bbox.json',
 'proposal': './result/retinanet-trt-ort.bbox.json'}

In [5]:
# result_path = "/home/geng/tinyml/ppq/benchmark/detection/result/retinanet-trt-ort.bbox.json"
result_path = "/home/geng/tinyml/ppq/benchmark/detection/OpenVino_output/Retinanet-PPQ-INT8.bbox.json"
report = dataset.evaluate(results_json_path=result_path)  
# dataset.evaluate(results=results) # 直接使用输出结果进行测试  


Evaluating bbox...
Loading and preparing results...
DONE (t=1.31s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


In [ ]:
dict(report)["map"]

In [1]:
import torch

In [2]:
box = torch.rand((100,4))
score = torch.rand((100,1))

In [5]:
result = torch.cat((box,score),dim=1)

In [7]:
result.shape

torch.Size([100, 5])

In [1]:
import fiftyone
z = fiftyone.zoo

In [ ]:
z.load_zoo_dataset("coco-2017")